In [12]:
from tqdm import tqdm_notebook as tqdm
from cfb import models as cfb_models
import pandas as pd; DF=pd.DataFrame
import numpy as np; import requests
import os
from django_pandas.io import read_frame as RF
from os.path import join
pwd=os.getcwd()
from utils import *
import warnings
warnings.filterwarnings("ignore")

In [13]:
# Create Season Instances
for year in range(1900,2100):
    season,c = Season.objects.get_or_create(
        year=year,defaults=dict(year=year)
    )

## Get Grad Rates



In [7]:
path = join(pwd,f'z_data\Graduation_Success_Rate.html')
df=pd.read_html(path)[0]

df['School']=df['School'].str.replace('University of','').str.strip()
df['School']=df['School'].str.replace('University','').str.strip()
df['School']=df['School'].str.replace(' at ','-').str.strip()

def match_teams(name):
    if name in team_match_dict.keys():
        return team_match_dict[name]
    else:
        return name
df['School']=df['School'].map(match_teams)


In [8]:
rdf=RF(School.objects.values('name'))
rdf=rdf.set_index('name')
jdf=rdf.join(df.set_index('School'))  

missing=jdf[pd.isna(jdf['Sport'])]
missing

,Cohort Year,Conference,Sport,State,GSR,FGR,GSR Report,FGR Report
Arkansas State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bowling Green,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Charlotte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fresno State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Georgia Tech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hawaii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Illinois,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Indiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df[df['School'].str.contains('Buffalo')].iloc[0].School

NameError: name 'df' is not defined

In [4]:
df['year']=df['Cohort Year']
display(df.sample(5).head())
df['School']=df['School'].str.replace('University of','').str.strip()
df['School']=df['School'].str.replace('University','').str.strip()
for tup in df.itertuples():
    season = Season.objects.get(year=tup.year)
    team_qs = Team.objects.filter(school__name=tup.School)
    if len(team_qs)==0:
        print(tup.School)
    
  

NameError: name 'df' is not defined

## Create Season/Coach/Conference Instances

In [7]:
path = join(pwd,'z_data\Coaches9.csv')
coaches_df=pd.read_csv(path)
col_list=['SchoolPay','TotalPay','Bonus','BonusPaid','AssistantPay','Buyout']
coaches_df = DataFrameCleaner(coaches_df).keep_numeric_only(col_list=col_list)


### Create Season Objects 
for year in range(1900,2100):
    season,c = Season.objects.get_or_create(
        year=year,defaults=dict(year=year)
    )

###  Create TeamSeason Objects for 2019
season = Season.objects.get(year=2019)
def main():
    for tup in tqdm(coaches_df.itertuples()):
        # get or create school
        school,c=cfb_models.School.objects.get_or_create(
            name=tup.School,defaults=dict(name=tup.School)
        )
        # get or create team
        team,c=cfb_models.Team.objects.get_or_create(
            school=school,type='Football',
            defaults=dict(school=school,type='Football')
        )
        # get or create conference
        conf,c=cfb_models.Conference.objects.get_or_create(
            name=tup.Conference,defaults=dict(name=tup.Conference)
        )

        # get or create coach
        coach_name_arr = tup.Coach.strip().split(' ')
        cfn=coach_name_arr[0]; cln=coach_name_arr[1]
        coach,c = cfb_models.Coach.objects.get_or_create(
            firstName=cfn,lastName=cln,
            defaults=dict(firstName=cfn,lastName=cln)
        )

        # get or create team season
        tseason,c = TeamSeason.objects.update_or_create(
            team=team,season=season,conference=conf,coach=coach,
            defaults=dict(team=team,season=season,conference=conf,coach=coach)
        )

        cpay,c = CoachPay.objects.update_or_create(
            teamseason=tseason,base=tup.SchoolPay,total=tup.TotalPay,
            bonus=tup.Bonus,bonusPaid=tup.BonusPaid,buyout=tup.Buyout,
            defaults=dict(
                teamseason=tseason,base=tup.SchoolPay,total=tup.TotalPay,
                bonus=tup.Bonus,bonusPaid=tup.BonusPaid,buyout=tup.Buyout,        
            )
        )

#main()  

## Create Team Season instances for previous seasons

In [5]:
def create_team_season_instances():
    for season in tqdm(Season.objects.all()):
        for team in Team.objects.all():
            teamseason,c = TeamSeason.objects.get_or_create(
                team=team,season=season,
                defaults=dict(team=team,season=season),
            )
            
#create_team_season_instances()

### Scrape cfbstats.com to extract url id for each team

In [6]:
#### Get Teams URL ID from cfbstats

def translate_team_name(old_name):
    team_match_dict={
        'BYU':'Brigham Young',"Hawai'i":'Hawaii', 
        'Miami (Florida)':'Miami (Fla.)', 'SMU':'Southern Methodist', 
        'TCU':'Texas Christian', 'UAB':'Alabama at Birmingham', 
        'UCF':'Central Florida', 'UNLV':'Nevada-Las Vegas',
        'USC':'Southern California','UTEP':'Texas-El Paso', 
        'UTSA':'Texas-San Antonio','Appalachian St.':'Appalachian State',
        'Fla. Atlantic':'Florida Atlantic',"Florida Int'l":'Florida International',
        'Western Ky.':'Western Kentucky','Middle Tenn. St.':'Middle Tennessee'
    } 
    old_name = replace_abbrevs(old_name)
    if old_name in team_match_dict.keys():
        new_name = team_match_dict[old_name]    
    else:
        new_name = old_name
    return new_name

def replace_abbrevs(old_name):
    new_name=old_name
    replace_arr=[
        ['St.','State'],
        ['Ala.','Alabama'],
        ['Ga.','Georgia'],
        ['Fla.','Florida'],
        ['Ill.','Illinois'],
        ['Caro.','Carolina'],
        ['Ky.','Kentucky'],
        ['La.','Louisiana'],
        ['Mich.','Michigan'],
        ['Miss.','Mississippi'],
    ]
    for r in replace_arr:
        new_name=new_name.replace(r[0],r[1])
    return new_name
    
def get_team_home_page_dfs_from_cfbstats(team_id,year):
    url = f'http://www.cfbstats.com/{year}/team/{team_id}/index.html'
    r=requests.get(url)
    try:
        dfs=pd.read_html(r.content)
    except ValueError:
        dfs=[]
    return dfs

def main():
    found_count=0
    target = len(coaches_df)
    for i in tqdm(range(0,1000)):
        urlId=str(i)
        dfs=get_team_home_page_dfs_from_cfbstats(i,year=2019)
        if len(dfs)==0: continue
        found_count+=1
        tname = dfs[0].columns[1]
        tname = translate_team_name(tname)
        team = Team.objects.filter(school__name=tname).first()

        cfbs_id,c = CfbstatsUrlId.objects.update_or_create(
            team=team,urlId=urlId,
            defaults=dict(team=team,urlId=urlId)
        )
#main()


## ----- Get 2019 Season Stats -----

In [10]:
def get_home_page_info(teamseason):
    dfs = get_team_home_page_dfs_from_cfbstats(teamseason.team.cfbs.urlId,year=teamseason.season.year)
    team_stats_df = dfs[0]
    team_stats_df = team_stats_df.rename(columns={'Unnamed: 0':'stat_str'})
    tname = team_stats_df.columns[1]
    team_stats_df = team_stats_df.rename(columns={tname:'value'})

    game_result_tdf = dfs[1]
    record_df = dfs[2]

    def get_loc(val):
        if val[0]=='@':
            loc = 'A' 
        elif val[0]=='+':
            loc = 'N'
        else:
            loc = 'H'
        return loc

    game_result_df = game_result_tdf.iloc[:-1,:].copy()
    game_result_df['loc']=game_result_df['Opponent'].map(get_loc)
    game_result_df['Opponent']=game_result_df['Opponent'].str.replace('@','').str.replace('+','')
    game_result_df['Date']=pd.to_datetime(game_result_df['Date'])
    game_result_df=game_result_df.rename(columns={'Game Time':'Game_Time'})

    for tup in record_df.itertuples():
        desc=tup.Split
        record = tup.Record.strip().split('-')
        win=record[0]
        loss=record[1]
        
        ts_record,c = TeamSeasonRecord.objects.update_or_create(
            teamseason=teamseason,desc=desc,
            defaults=dict(teamseason=teamseason,desc=desc,win=win,loss=loss)
        )
    for tup in game_result_df.itertuples():
        date = tup.Date
        attendance = tup.Attendance
        result_str = tup.Result
        result = result_str[0]
        if result not in ['L','W']: result = 'T'
        score_str = result_str[2:]
        score1 = score_str.split('-')[0]
        score2 = score_str.split('-')[1]
        score_arr = [int(score1),int(score2)]
        if result=='W':
            score,oscore=(max(score_arr),min(score_arr))
        else:
            score,oscore=(min(score_arr),max(score_arr))

        oname=tup.Opponent.strip()\
            .replace('St.','State').replace('Ala.','Alabama')
        if oname[0].isnumeric():
            orank = oname.split(' ')[0]
            oname = " ".join(oname.split(' ')[1:])
        else:
            orank=None
        oname = translate_team_name(oname)       

        glen = str(tup.Game_Time)
        glen_arr = glen.split(':')
        hours = float(glen_arr[0])
        hour_frac = float(glen_arr[1])/60 if len(glen.split(':'))>1 else 0
        glen = hours + hour_frac
        opponent = TeamSeason.objects.filter(
            season=teamseason.season,
            team__school__name=oname
        ).first()

        gr,c = GameResult.objects.update_or_create(
            teamseason=teamseason,
            oname=oname,
            date=date,
            defaults=dict(
                teamseason=teamseason,
                opponent=opponent,
                oname=oname,
                date=date,
                score=score,
                oscore=oscore,
                result=result,
                attendance=attendance,
                length=glen,
            )
        )
    for tup in team_stats_df.itertuples():
        stat_str = tup.stat_str
        category=(stat_str.split(':')[0]).replace('/','per')
        desc_str=(stat_str.split(':')[1]).strip() if len(stat_str.split(':'))>1 else category


        desc_arr = desc_str.replace('/','per').split(' - ')
        value_arr = tup.value.replace('%','').split(' - ')
        ovalue_arr = tup.Opponents.replace('%','').split(' - ')


        vtups = list(zip(desc_arr,value_arr,ovalue_arr))
        for vtup in vtups:
            desc=vtup[0]
            value=vtup[1]
            ovalue=vtup[2]
            value = value.replace(',','')
            ovalue = ovalue.replace(',','')  
            if ':' in value:
                value = float(value.split(':')[0]) + float(value.split(':')[1])/60
                ovalue = float(ovalue.split(':')[0]) + float(ovalue.split(':')[1])/60

            if value=='-':value=np.nan
            if ovalue=='-':ovalue=np.nan
            tss,c = TeamSeasonStat.objects.update_or_create(
                teamseason=teamseason,
                category=category,
                desc=desc,
                defaults=dict(
                    teamseason=teamseason,
                    category=category,
                    desc=desc,
                    value=value,
                    ovalue=ovalue               
                )
            )
            
for year_num in tqdm(range(2010,2020)):
    season = Season.objects.get(year=year_num)
    print(season)
    for teamseason in tqdm(TeamSeason.objects.filter(season=season)):
        try:
            get_home_page_info(teamseason)
        except IndexError:
            pass
    

2010



2011



2012



2013



2014



2015



2016



2017



2018



2019


In [9]:
def get_stadium_capacities(show_results=True):
    url=r'https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_FBS_football_stadiums'
    df=pd.read_html(url)[0]
    df=df.rename(columns={'Record1':'att_record','Expanded 2':'Expanded'})
    df=df.drop('Image',axis=1)
    def remove_wiki_citation(df,col_list=None):
        if col_list == None:
            col_list = df.columns
        for col in col_list:
            df[col]=df[col].str.replace('[\[\d\]]]','').str.replace(r'[','')

    DFC = DataFrameCleaner(df)
    DFC.remove_wiki_citation()
    DFC.keep_numeric_only(col_list=['Capacity','Built','Expanded'],type_coerce=np.float)
    df = DFC.df
    df=df.replace(np.nan,None)
    df['Built']=df['Built'].astype(str).str[:4].astype(int)
    df['Expanded']=df['Expanded'].astype(str).str[:4].astype(int)
    for tup in df.itertuples():
        stadium,c = Stadium.objects.update_or_create(
            name=tup.Stadium,
            defaults=dict(
                name=tup.Stadium,
                city=tup.City,
                state=tup.State,
                year_built=tup.Built,
                year_expanded=tup.Expanded,
                capacity=tup.Capacity,
                wiki_team_name=tup.Team,
                surface=tup.Surface
            )
        )
        school_name = stadium.wiki_team_name
        if stadium.wiki_team_name == 'Miami':
            school_name = 'Miami (Fla.)' if tup.State=='OH' else 'Miami (Ohio)'
        if stadium.wiki_team_name == 'Louisiana':
            school_name = 'Louisiana-Lafayette'
        if stadium.wiki_team_name[:5] == 'Hawai':
            school_name = 'Hawaii'  

        team = Team.objects.filter(school__name=school_name).first()
        if pd.isna(team):
            school_name = team_match_dict.get(school_name)
            team = Team.objects.filter(school__name=school_name).first()

        teamseason_qs = team.teamseason_set.filter(season__year__gte=stadium.year_built)
        if len(teamseason_qs)==0:
            print(stadium)
            print(stadium.year_built)
            print(team)
        if show_results:
            print(stadium)
#get_stadium_capacities(show_results=True) 